# Portugal

This notebook contains all the Julia 1.4.x code required to characterize the 
propagation of CoViD19 as discussed in the main notebook:
 _Population_dynamics.ipynb_

Here we use simple _logistic_ like models, based only on the total number of 
infected cases, for more robust and elaborate models consider the SIR_like 
notebooks found in this folder.

### Initialization, display LOAD_PATH at the end

In [ ]:
push!(LOAD_PATH, pwd())
if ispath( pwd()*"/src")
    push!(LOAD_PATH, pwd()*"/src")
end


using SpecialFunctions, LaTeXStrings
using DataFrames, Query, CSV, Dates
using LsqFit

using MyFunctions, Mrate

using Plots
# plotly();
theme( :gruvbox_light );
mysize = ( Int( round( 400 *MathConstants.golden ) ), 400 );

### Parametrization for the incubation times, source:
https://doi.org/10.2807/1560-7917.ES.2020.25.5.2000062

In [ ]:
μΓ, σΓ = 6.5, 2.6;      # mean and standard deviation for incubation

αΓ( μ, σ ) = μ^2/σ^2;   # [α] = 1  is a dimensionless parameter
βΓ( μ, σ ) = μ / σ^2    # [β] = 1/T has dimension of frequency

α0 = αΓ( μΓ, σΓ );
β0 = βΓ( μΓ, σΓ );

ρ0=[α0,β0];

### load data and manipulate data

In [ ]:
path = pwd()*"/datahub.io/covid-19/data";
path_data = path*"/time-series-19-covid-combined.csv";
path_ref  = path*"/reference.csv";

raw_data = CSV.File( path_data ) |> DataFrame!;
col_names = names( raw_data )

col_names[2] = "CountryRegion";
col_names[3] = "ProvinceState";
rename!( raw_data, col_names );

In [ ]:
country_names =  unique( raw_data[!,2] );
country_name = "Portugal";
country = raw_data |> @filter( _.CountryRegion==country_name
    ) |> DataFrame;

In [ ]:
dates = Date.( country[!,:Date] );

confirmed_int = convert( Array{Number}, country[!,:Confirmed] );
recovered_int = convert( Array{Number}, country[!,:Recovered] );
deaths_int    = convert( Array{Number}, country[!,:Deaths   ] );

In [ ]:
d1   = findfirst( >=(1  ), confirmed_int );
d10  = findfirst( >=(10 ), confirmed_int );
d20  = findfirst( >=(20 ), confirmed_int );
d50  = findfirst( >=(50 ), confirmed_int );
d100 = findfirst( >=(100), confirmed_int );

read raw data and basic processing

In [ ]:
raw_ref  = CSV.File( path_ref  ) |> DataFrame!;
ref_names = names( raw_ref )

IDs = convert( Vector, raw_ref[!,11] );

row_country = findall( isequal( country_name ), IDs );
@assert length( row_country ) == 1
population = convert.( Number, raw_ref[ row_country[1], 12 ] );

println( [ population, 1/population ]' )

In [ ]:
#   set initial time and offset it to day of first registered infection
date_1 = dates[ d1 ]
data_last = dates[ end ]
days = float.( Dates.value.( dates -date_1 ) )

# normalize population
confirmed = confirmed_int ./ population;
recovered = recovered_int ./ population;
deaths    = deaths_int    ./ population;

# determine normalized sir time series
removed     = ( recovered +deaths  );
infected    = ( confirmed -removed );
susceptible = ( 1 .-infected .-removed );

#### visualize raw data

In [ ]:
nM = 1.25 *maximum( confirmed )
nm = 0.75 *minimum( confirmed )


tm, tM = days[d1], maximum( days ) +5;

In [ ]:
plot( days, confirmed, label="total infected"
    , shape=:cross, msize=5, lα=0
    , xlabel = L"t \, \mathrm{(d)}", xlims = (0,tM), xticks = 0:15:tM
    , ylabel = L"n \, (\mathrm{cases}/N_t)", ylims = (-500*nm,nM)
    , legend=:topleft, legendfontsize=6, size=mysize )
plot!( days, infected, label="infected"
    , markershape=:xcross, markersize=5, linealpha=0 )
plot!( days, recovered, label="recovered"
    , markershape=:hline, markersize=5, linealpha=0 )
plot!( days, deaths, label="deaths"
    , markershape=:vline, markersize=5, linealpha=0 )

#### Note the large jump at  t=83

On this day, 2020-05-24, the Portuguese authorities change the criteria that 
defined the transition from infected to recovered status, leading to a large 
conversion of infected to recovered cases.

This in criteria breaks the characterization of the infected and recovered time 
series, but it is not relevant for the current model, which only considers the 
total number of infections, rather than the _active_ infected cases.

Please refer to the discussion on the limitations of the _logistic_ model and 
see the generalization to the SIR like models for further details.

Consider the following reports in the Portuguese press
[Público](https://www.publico.pt/2020/05/24/sociedade/noticia/nova-contagem-casos-traz-recorde-recuperados-covid19-1917895)
and
[Sabado](https://www.sabado.pt/portugal/detalhe/covid-19-numero-de-recuperados-vai-disparar-avisa-ministra).

In [ ]:
#
# https://www.sabado.pt/portugal/detalhe/covid-19-numero-de-recuperados-vai-disparar-avisa-ministra
t_break_1 = findfirst( ==(Dates.Date("2020-05-24")), dates )
println( t_break_1 -d1, "  ", dates[ t_break_1 ] )

In [ ]:
plot!( yscale = :log10, ylims = (5e-7,5e-3) )

In [ ]:
plot!( yscale = :auto, ylims = (-500*nm,nM) );

### The models
__Note:__ parameter p[2] is not multiplied by p[1] as in the main notes. Here p[2] is dimensionless! To recover the definition used in the main text divide by p[1], ie t_0 = p[2]/p[1]

In [ ]:
# simple logistic model
@. model0(t,p) = 1/( 1.0 +exp( -p[1]*t +p[2] ) );

In [ ]:
# the m-steady state model and its Jacobian
# definitions of modelm, modelm_j and all other relevant functions
# at Mrate.jl and MyFunctions.jl

function m_m( t, p )
    modelm(t,p,ρ0)
end;

function j_m(t::Array,p)
    J = Array{Float64}(undef, length(t), length(p))
    J = modelm_j(t, p, ρ0)
end;

In [ ]:
p0 = [0.25, 0. ];
x0 = [0, 1.0, 2.0];
model0(0.0, p0);
model0(x0, p0);
m_m(0.0,p0);
m_m(x0,p0);
j_m(x0,p0);

### Nonlinear fits
#### The logistic fit

In [ ]:
t_offset, d_off = date_1, d1
tcont = range(days[1],days[end]*1.1; length=3*length(days))

dm, dM = d1, length(days);
xdata = days[dm:dM]; ydata = confirmed[dm:dM];


fit0 = curve_fit( model0, xdata, ydata, p0 )
estimate0 = model0( tcont, fit0.param )

# half population time and date
t12 = floor( fit0.param[2]/fit0.param[1] )
t_offset +Dates.Day( t12 )

plot( days, confirmed, label="total infected"
    , shape=:cross, msize=5, lα=0
    , xlabel = L"t \, \mathrm{(d)}", xlims = (0,tM), xticks = 0:15:tM
    , ylabel = L"n \, (\mathrm{cases}/N_t)", ylims = (-500*nm,nM)
    , legend=:topleft, legendfontsize=6, size=mysize )

plot!( tcont, estimate0, label="logistic", linewidth=1.5 )

In [ ]:
# ------------------------------------ #
#
#   truncated logistic 1
# 
dm, dM = d_off, d_off+30; p0 = [1., 25]
xdata = days[dm:dM]; ydata = confirmed[dm:dM];

fit0b = curve_fit( m_m, j_m, xdata, ydata, p0; )
estimate0b = m_m( tcont, fit0b.param )

println( round.( fit0b.param'; sigdigits=3 )  )

# half population time and date
t12 = floor( fit0b.param[2]/fit0b.param[1] )
t_offset +Dates.Day( t12 )


# ------------------------------------ #
#
#   truncated logistic 2
# 
dm, dM = d_off+30, d_off+55;
xdata = days[dm:dM]; ydata = confirmed[dm:dM];

fit0c = curve_fit( m_m, j_m, xdata, ydata, p0 )
estimate0c = m_m( tcont, fit0c.param )


fit0c.param


# ------------------------------------ #
#
#   truncated logistic 3
# 
dm, dM = d_off+60, length(days);
xdata = days[dm:dM]; ydata = confirmed[dm:dM];

fit0d = curve_fit( m_m, j_m, xdata, ydata, [0.001, 10.] )
estimate0d = m_m( tcont, fit0d.param )



plot!( tcont, estimate0b, linewidth = 1.5, label="truncated logistic"  )
plot!( tcont, estimate0c, linewidth = 1.5, label="truncated logistic 2" )
plot!( tcont, estimate0d, linewidth = 1.5, label="truncated logistic 3" )

#### two steady state model m=1

In [ ]:
# ------------------------------------ #
# 
#   two rate steady state m = 1
# 
dm, dM = d_off, length(days); p1 = [0.2, 15, 0.05, 50 ];
xdata = days[dm:dM]; ydata = confirmed[dm:dM];

fit1 = curve_fit( m_m, j_m, xdata, ydata, p1 )
estimate1 = m_m( tcont, fit1.param )

println( round.( fit1.param'; sigdigits=3 )  )


# ------------------------------------ #
# 
#   truncated two rate steady state m = 1
# 
dm, dM = d_off, d_off+55; p1 = [0.2, 15, 0.05, 16 ];
xdata = days[dm:dM]; ydata = confirmed[dm:dM];

fit1b = curve_fit( m_m, j_m, xdata, ydata, p1 )
estimate1b = m_m( tcont, fit1b.param )

fit1b.param[2]/fit1b.param[1]
t12 = floor( fit1b.param[2]/fit1b.param[3] )
t_offset +Dates.Day( t12 )
t_offset +Dates.Day( floor( fit1b.param[4] ) )

println( round.( fit1b.param'; sigdigits=3 )  )


# transition date (day)
println( t_offset +Dates.Day.( floor.( [ fit1.param[4],fit1b.param[4] ] )) )


plot( days, confirmed, label="total infected"
    , shape=:cross, msize=5, lα=0
    , xlabel = L"t \, \mathrm{(d)}", xlims = (0,tM), xticks = 0:15:tM
    , ylabel = L"n \, (\mathrm{cases}/N_t)", ylims = (-500*nm,nM)
    , legend=:topleft, legendfontsize=6, size=mysize )
plot!( tcont, estimate1, linewidth=1.5
    , label="two steady states" )

plot!( tcont, estimate1b, linewidth=1.5
    , label="truncated two steady states" )

#### three steady state m=2

In [ ]:
# ------------------------------------ #
# 
#   three rate steady state m = 1
# 
dm, dM = d_off, length(days); p2 = [0.25, 15, 0.01, 20, 0.005, 55 ]
xdata = days[dm:dM]; ydata = confirmed[dm:dM];

fit2 = curve_fit( m_m, j_m, xdata, ydata, p2 )
estimate2 = m_m( tcont, fit2.param );
println( round.( fit2.param'; sigdigits=3 ) )

# ------------------------------------ #
# 
#   constrained three rate steady state m = 1
# 
lb = [ 0.1, 1   , 0.005, 20, 0.00001, 45 ];
ub = [ 0.3, 1000, 0.1  , 45, 0.1    , 60 ];
fit2b = curve_fit( m_m, j_m, xdata, ydata, p2; lower=lb, upper=ub )

estimate2b = m_m( tcont, fit2b.param )
println( round.( fit2b.param'; sigdigits=3 ) )


plot( days, confirmed, label="total infected"
    , shape=:cross, msize=5, lα=0
    , xlabel = L"t \, \mathrm{(d)}", xlims = (0,tM), xticks = 0:15:tM
    , ylabel = L"n \, (\mathrm{cases}/N_t)", ylims = (-500*nm,nM)
    , legend=:topleft, legendfontsize=6, size=mysize )
plot!( tcont, estimate2, linewidth=1.5, label="three steady states" )
plot!( tcont, estimate2b, linewidth=1.5, label="constrained three steady states" )

In [ ]:
# transition dates
println( t_offset +Dates.Day.( floor.( fit2b.param[4:2:length(p2)]' ) ) )

In [ ]:
# doubling times for each rate

println( round.( log(2) ./fit2b.param[1:2:length(p2)]'; sigdigits=3 ) )

# Combined plot and fit quality

In [ ]:
plot( days, confirmed, label="total infected"
    , shape=:cross, msize=5, lα=0
    , xlabel = L"t \, \mathrm{(d)}", xlims = (0,tM), xticks = 0:15:tM
    , ylabel = L"n \, (\mathrm{cases}/N_t)", ylims = (-500*nm,nM)
    , legend=:bottomright, legendfontsize=6, size=mysize )

plot!( tcont, estimate0, label="logistic", linewidth=1.5 )

plot!( tcont, estimate0b, linewidth = 1.5, label="truncated logistic"  )
plot!( tcont, estimate0c, linewidth = 1.5, label="truncated logistic 2" )
plot!( tcont, estimate0d, linewidth = 1.5, label="truncated logistic 3" )

plot!( tcont, estimate1b, linewidth=1.5, label="truncated two steady states" )

plot!( tcont, estimate2, linewidth=1.5, label="three steady states" )

In [ ]:
savefig("img/"*country_name*"_fits_converged.svg")
plot!( yscale = :log10, ylims = (5e-7, 5-3) )

In [ ]:
savefig("img/"*country_name*"_fits_converged_log.svg")
plot!( yscale = :auto, ylims = (nm,nM) );

##### Estimate quality of fit

In [ ]:
cov_mat = estimate_covar(fit2)
sigma = stderror(fit2)

margin_of_error = margin_error(fit2, 0.05)

aux = confidence_interval(fit2, 0.05)
confidence_inter = zeros(2,6)
for i=1:length(p2)
    confidence_inter[:,i] = collect( aux[i] )
end


# display the parameters
println( round.( fit2.param; sigdigits=3 )' )

In [ ]:
# estimate the margins of error
println( round.( sigma; sigdigits=3 )' )

In [ ]:
# estimate the margins of error
println( round.( margin_of_error; sigdigits=3 )' )

In [ ]:
# estimate the margins of error
round.( confidence_inter; sigdigits=3 )

In [ ]:
# the covariance matrix estimate
round.( cov_mat; sigdigits=3 )